# <font color="#418FDE" size="6.5" uppercase>**Tool Calling Basics**</font>

>Last update: 20260118.
    
By the end of this Lecture, you will be able to:
- Explain how LangChain exposes tools for Llama 3 to call during reasoning. 
- Define and register simple tools in LangChain and connect them to a Llama 3 model. 
- Test and debug tool-using interactions to ensure correct tool selection and safe outputs. 


## **1. Core Tool Use Concepts**

### **1.1. When Llama 3 needs tools**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_04/Lecture_B/image_01_01.jpg?v=1768769323" width="250">



>* Model handles text reasoning using learned knowledge
>* Needs tools for live data and exact operations

>* Model decides on tools during stepwise reasoning
>* Uses tools for fresh data and specialized tasks

>* Complex planning questions often trigger helpful tool use
>* Model leads reasoning while tools handle precise operations



### **1.2. Tool Call Schema Essentials**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_04/Lecture_B/image_01_02.jpg?v=1768769340" width="250">



>* Schemas describe each tool’s purpose and inputs
>* Clear schemas let Llama 3 choose tools reliably

>* Schema defines tool purpose and input parameters
>* Standardized schema lets Llama 3 call tools reliably

>* Schemas guide multi-step tool choices and calls
>* LangChain runs tools and returns results as context



In [ ]:
#@title Python Code - Tool Call Schema Essentials

# Demonstrate simple tool schema as shared contract between model and tools.
# Show how schema describes tool name, description, and input parameters.
# Simulate model reading schema and building structured tool call arguments.

# pip install langchain-openai langchain-community langchain.

# Define a simple tool schema as a Python dictionary.
weather_tool_schema = {
    "name": "get_weather_fahrenheit",
    "description": "Get current weather in Fahrenheit for a given US city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Name of the US city to check.",
            },
            "unit": {
                "type": "string",
                "description": "Temperature unit, use Fahrenheit only.",
            },
        },
        "required": ["city", "unit"],
    },
}

# Print the schema so we can see the shared contract.
print("Tool schema contract:")
print({"name": weather_tool_schema["name"], "parameters": weather_tool_schema["parameters"]["properties"]})

# Define a simple user request that needs weather information.
user_request = "What is the temperature in Boston right now in Fahrenheit?"

# Simulate model reasoning by matching request words with schema description.
if "Fahrenheit" in user_request and "temperature" in user_request:
    chosen_tool_name = weather_tool_schema["name"]
    tool_arguments = {"city": "Boston", "unit": "Fahrenheit"}
else:
    chosen_tool_name = None
    tool_arguments = {}

# Show the structured tool call that follows the schema exactly.
print("\nStructured tool call:")
print({"tool": chosen_tool_name, "arguments": tool_arguments})

# Simulate tool execution using the structured arguments from the schema.
def fake_get_weather_fahrenheit(city, unit):
    return f"Weather in {city} is 72 degrees {unit} right now."

# Execute the fake tool only if a tool was chosen.
if chosen_tool_name:
    result = fake_get_weather_fahrenheit(**tool_arguments)
else:
    result = "No matching tool was selected by the reasoning process."

# Show final answer that the model could return to the user.
print("\nFinal answer using tool:")
print(result)



### **1.3. Safe Tool Usage**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_04/Lecture_B/image_01_03.jpg?v=1768769384" width="250">



>* Each tool adds power but also risk
>* Grant tools like you grant employee permissions

>* Careful tool descriptions teach correct, safe usage
>* Schemas and validation guard against risky tool calls

>* Monitor and log tool calls to spot misuse
>* Use feedback to refine tools, balancing safety



In [ ]:
#@title Python Code - Safe Tool Usage

# Demonstrate safe tool usage concepts with simple Python functions and validation.
# Show difference between safe read tools and risky write tools clearly.
# Log and block unsafe requests before executing risky operations safely.
# !pip install some_required_library_here if external libraries were actually needed.

# Define a simple read only tool that is always safe.
def safe_lookup(zip_code, miles_radius):
    return f"Searching public stores within {miles_radius} miles of ZIP {zip_code}."

# Define a risky write tool that can place real orders.
def risky_place_order(user_id, item_name, quantity):
    return f"ORDER PLACED for {quantity} {item_name} for user {user_id}."

# Define a safety checker that validates risky tool arguments.
def validate_order_request(user_authenticated, quantity):
    if not user_authenticated:
        return False, "Blocked: user not authenticated for placing orders."
    if quantity <= 0 or quantity > 5:
        return False, "Blocked: quantity must be between one and five units."
    return True, "Order request passed safety checks."

# Simulate a model deciding which tool to call unsafely.
def model_decision_simulation(user_message):
    if "buy" in user_message.lower():
        return "risky_place_order"
    return "safe_lookup"

# Simulate a safe orchestrator that wraps tool calls with checks.
def safe_orchestrator(user_message, user_authenticated):
    print("User message:", user_message)
    chosen_tool = model_decision_simulation(user_message)
    print("Model chose tool:", chosen_tool)
    if chosen_tool == "safe_lookup":
        result = safe_lookup("10001", 10)
        print("Tool result:", result)
        return
    is_ok, reason = validate_order_request(user_authenticated, quantity=10)
    print("Safety check:", reason)
    if not is_ok:
        print("Final action: risky tool call blocked safely.")
        return
    result = risky_place_order("user_123", "flashlight", 10)
    print("Tool result:", result)

# Run two scenarios showing unsafe model choice but safe wrapper behavior.
print("Scenario one: unauthenticated user tries large purchase.")
safe_orchestrator("Please buy ten flashlights for me.", user_authenticated=False)
print("\nScenario two: simple store lookup request.")
safe_orchestrator("Where can I buy a flashlight near me?", user_authenticated=True)



## **2. Defining LangChain Tools**

### **2.1. Python Function Tools**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_04/Lecture_B/image_02_01.jpg?v=1768769418" width="250">



>* Tools are just focused Python functions for tasks
>* LangChain lets Llama 3 call these functions

>* Design single-purpose functions with clear, structured inputs
>* Model selects tools, functions hold domain logic

>* Give tools clear names and concise descriptions
>* Good documentation improves tool choice and reasoning



In [ ]:
#@title Python Code - Python Function Tools

# Demonstrate simple Python function tools for LangChain style usage.
# Show clear inputs and outputs for predictable tool behavior.
# Print example tool calls and their structured results.
# pip install langchain-openai langchain-community langchain.

# Define a simple temperature conversion tool function.
def convert_fahrenheit_to_celsius(fahrenheit_value: float) -> float:
    """Convert Fahrenheit temperature to Celsius temperature value."""
    celsius_value = (fahrenheit_value - 32.0) * 5.0 / 9.0
    return round(celsius_value, 2)

# Define a simple loan payment estimation tool function.
def estimate_monthly_payment(principal_dollars: float, annual_rate_percent: float, years_term: int) -> float:
    """Estimate fixed monthly payment for simple loan."""
    monthly_rate = annual_rate_percent / 100.0 / 12.0
    months_total = years_term * 12
    if monthly_rate == 0:
        payment_value = principal_dollars / months_total
    else:
        factor_value = (1 + monthly_rate) ** months_total
        payment_value = principal_dollars * monthly_rate * factor_value / (factor_value - 1)
    return round(payment_value, 2)

# Define a dispatcher that mimics a language model choosing tools.
def call_tool(tool_name: str, **kwargs) -> dict:
    """Call selected tool function using keyword arguments."""
    if tool_name == "convert_fahrenheit_to_celsius":
        result_value = convert_fahrenheit_to_celsius(kwargs["fahrenheit_value"])
    elif tool_name == "estimate_monthly_payment":
        result_value = estimate_monthly_payment(
            kwargs["principal_dollars"], kwargs["annual_rate_percent"], kwargs["years_term"]
        )
    else:
        result_value = "Unknown tool requested by caller."
    return {"tool": tool_name, "inputs": kwargs, "result": result_value}

# Demonstrate calling the temperature conversion tool.
weather_tool_call = call_tool("convert_fahrenheit_to_celsius", fahrenheit_value=77.0)
print("Weather tool call result:", weather_tool_call)

# Demonstrate calling the loan payment estimation tool.
loan_tool_call = call_tool(
    "estimate_monthly_payment", principal_dollars=20000.0, annual_rate_percent=5.0, years_term=5
)
print("Loan tool call result:", loan_tool_call)



### **2.2. Tool Input Schemas**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_04/Lecture_B/image_02_02.jpg?v=1768769444" width="250">



>* Tool input schemas clearly define expected arguments
>* They reduce ambiguity and simplify debugging tool calls

>* Schemas guide model choices and document tools
>* They ensure complete, coherent, valid tool calls

>* Schemas restrict inputs to safe, sensible values
>* Framework validation catches errors and improves reliability



In [ ]:
#@title Python Code - Tool Input Schemas

# Demonstrate simple tool input schemas using Python dictionaries and validation logic.
# Show how required and optional fields can be checked before tool execution.
# Print validation results to illustrate correct and incorrect tool calls.

# pip install pydantic langchain-openai langchain-community langchain.

# Define a simple weather tool input schema using a dictionary template.
weather_schema = {
    "city": {"type": "string", "required": True},
    "date": {"type": "string", "required": False},
}

# Define a simple distance tool input schema using another dictionary template.
distance_schema = {
    "from_city": {"type": "string", "required": True},
    "to_city": {"type": "string", "required": True},
}

# Define a function that validates arguments against a provided schema dictionary.
def validate_inputs(schema, arguments):
    errors = []
    for field_name, field_rules in schema.items():
        if field_rules["required"] and field_name not in arguments:
            errors.append(f"Missing required field: {field_name}")
        if field_name in arguments and not isinstance(arguments[field_name], str):
            errors.append(f"Field {field_name} must be string type")
    for provided_name in arguments.keys():
        if provided_name not in schema:
            errors.append(f"Unexpected field provided: {provided_name}")
    return errors

# Define a function that simulates calling a weather tool after validation.
def call_weather_tool(arguments):
    errors = validate_inputs(weather_schema, arguments)
    if errors:
        print("Weather tool call invalid due to these issues:")
        for issue in errors:
            print("-", issue)
    else:
        city = arguments.get("city")
        date = arguments.get("date", "today")
        print(f"Weather tool called successfully for {city} on {date}.")

# Define a function that simulates calling a distance tool after validation.
def call_distance_tool(arguments):
    errors = validate_inputs(distance_schema, arguments)
    if errors:
        print("Distance tool call invalid due to these issues:")
        for issue in errors:
            print("-", issue)
    else:
        start = arguments.get("from_city")
        end = arguments.get("to_city")
        print(f"Distance tool called successfully from {start} to {end}.")

# Prepare a valid weather tool call with required and optional fields included.
valid_weather_call = {"city": "Chicago", "date": "2026-07-04"}

# Prepare an invalid weather tool call missing the required city field.
invalid_weather_call = {"date": "2026-07-04"}

# Prepare a valid distance tool call with both required cities provided.
valid_distance_call = {"from_city": "Boston", "to_city": "New York"}

# Prepare an invalid distance tool call with an unexpected extra argument.
invalid_distance_call = {"from_city": "Dallas", "to_city": "Houston", "miles": 250}

# Execute all tool calls and observe how schemas guide validation behavior.
if __name__ == "__main__":
    print("Valid weather call result:")
    call_weather_tool(valid_weather_call)
    print("\nInvalid weather call result:")
    call_weather_tool(invalid_weather_call)
    print("\nValid distance call result:")
    call_distance_tool(valid_distance_call)
    print("\nInvalid distance call result:")
    call_distance_tool(invalid_distance_call)



### **2.3. Registering Tools in Chains**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_04/Lecture_B/image_02_03.jpg?v=1768769475" width="250">



>* Register tools in a chain for use
>* Chain lets Llama 3 choose and sequence tools

>* Shape system instructions to describe tools and constraints
>* Clear guidance ensures safe, predictable tool use

>* Chains coordinate multiple tools for complex workflows
>* Unified interface scales as you add tools



In [ ]:
#@title Python Code - Registering Tools in Chains

# Demonstrate registering simple tools inside a LangChain style conversation chain.
# Show how a model chooses between registered tools during reasoning steps.
# Print final answer plus which tools were selected and called.

# pip install langchain langchain-openai langchain-community.
# pip install python-dotenv.

# Import required standard and external libraries.
from typing import List, Dict, Any
from dataclasses import dataclass

# Define a simple fake language model that decides tool usage.
@dataclass
class FakeLlamaModel:
    name: str

# Define a simple tool interface holding name and callable function.
@dataclass
class SimpleTool:
    name: str
    description: str
    func: Any

# Define a chain that holds model, tools, and system instructions.
@dataclass
class ToolEnabledChain:
    model: FakeLlamaModel
    tools: List[SimpleTool]
    system_instructions: str

# Define a method that routes user questions to appropriate tools.
    def run(self, user_message: str) -> Dict[str, Any]:
        chosen_tools: List[str] = []
        answer_parts: List[str] = []

        if "inventory" in user_message.lower() or "stock" in user_message.lower():
            inventory_tool = self._get_tool_by_name("inventory_lookup")
            if inventory_tool is not None:
                chosen_tools.append(inventory_tool.name)
                answer_parts.append(inventory_tool.func("flashlight"))

        if "shipping" in user_message.lower() or "delivery" in user_message.lower():
            shipping_tool = self._get_tool_by_name("shipping_estimate")
            if shipping_tool is not None:
                chosen_tools.append(shipping_tool.name)
                answer_parts.append(shipping_tool.func("90210"))

        if not answer_parts:
            answer_parts.append("I can answer without calling any registered tools.")

        final_answer = " ".join(answer_parts)
        return {"answer": final_answer, "used_tools": chosen_tools}

# Helper method that finds a tool by its registered name.
    def _get_tool_by_name(self, name: str) -> Any:
        for tool in self.tools:
            if tool.name == name:
                return tool
        return None

# Define a simple inventory lookup tool function.
def inventory_lookup_tool(item_name: str) -> str:
    inventory_data = {"flashlight": 12, "tent": 5, "compass": 0}
    quantity = inventory_data.get(item_name, 0)
    if quantity > 0:
        return f"Inventory tool reports {quantity} units of {item_name} currently in stock."
    return f"Inventory tool reports {item_name} currently out of stock at all warehouses."

# Define a simple shipping estimate tool function using miles distance.
def shipping_estimate_tool(zip_code: str) -> str:
    base_distance_miles = 1200
    days = 3 if zip_code.startswith("9") else 5
    return f"Shipping tool estimates delivery in {days} days over approximately {base_distance_miles} miles."

# Create tool objects that wrap the Python functions and descriptions.
inventory_tool = SimpleTool(
    name="inventory_lookup",
    description="Check current warehouse inventory for a specific camping product.",
    func=inventory_lookup_tool,
)

# Create a second tool for shipping estimates and register description.
shipping_tool = SimpleTool(
    name="shipping_estimate",
    description="Estimate shipping time in days for a given United States zip code.",
    func=shipping_estimate_tool,
)

# Create a fake Llama model instance representing our reasoning engine.
model = FakeLlamaModel(name="FakeLlama-Tool-Demo")

# Define system instructions that explain how tools should be used.
system_message = (
    "You are a helpful store assistant. Use inventory_lookup for stock questions, "
    "and shipping_estimate for delivery questions. Avoid tools for casual chit chat."
)

# Register tools and model together inside a single tool enabled chain.
chain = ToolEnabledChain(model=model, tools=[inventory_tool, shipping_tool], system_instructions=system_message)

# Define a user question that naturally requires both registered tools.
user_question = "Is the flashlight in stock and how long is shipping to zip 90210?"

# Run the chain, letting it decide which tools to call and in which order.
result = chain.run(user_question)

# Print the final answer plus which tools were actually used.
print("User question:", user_question)
print("Final answer:", result["answer"])
print("Tools used:", ", ".join(result["used_tools"]))



## **3. Testing LangChain Tools**

### **3.1. Tracing Tool Calls**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_04/Lecture_B/image_03_01.jpg?v=1768769514" width="250">



>* Tracing reveals each step of tool usage
>* Helps diagnose wrong tools, bad arguments, ignored outputs

>* Run realistic chats, then inspect every tool step
>* Use traces to catch bad chaining or hallucinations

>* Use traces to monitor safety, cost, latency
>* Detect data leaks, regressions, and unhealthy tool usage



In [ ]:
#@title Python Code - Tracing Tool Calls

# Demonstrate simple LangChain style tool tracing behavior clearly.
# Show how a model might decide and call tools stepwise.
# Print a compact trace for easy beginner friendly understanding.

# pip install langchain-openai langchain-community langchain.

# Import required standard libraries for simple tracing simulation.
import json
import random

# Define a simple tool registry with two example tools available.
tools = {
    "get_weather": "Fetches current weather for a given city.",
    "get_time": "Fetches current local time for a given city."
}

# Define a function that simulates model tool selection decisions.
def choose_tool(user_message):
    if "weather" in user_message.lower():
        return "get_weather"
    if "time" in user_message.lower():
        return "get_time"
    return None

# Define a function that simulates calling a selected tool safely.
def call_tool(tool_name, city):
    if tool_name == "get_weather":
        return {"city": city, "temperature_f": 72, "condition": "sunny"}
    if tool_name == "get_time":
        return {"city": city, "local_time": "03:15 PM"}
    return {"error": "unknown tool"}

# Define a function that runs conversation and records detailed trace.
def run_with_trace(user_message, city):
    trace = []
    trace.append({"event": "user_message", "content": user_message})
    tool_name = choose_tool(user_message)
    trace.append({"event": "tool_chosen", "tool": tool_name})
    if tool_name is not None:
        args = {"city": city}
        trace.append({"event": "tool_call", "tool": tool_name, "args": args})
        result = call_tool(tool_name, city)
        trace.append({"event": "tool_result", "tool": tool_name, "result": result})
        final_answer = f"In {city}, here is the tool based information: {result}."
    else:
        final_answer = "I will answer directly without using any tools here."
    trace.append({"event": "final_answer", "content": final_answer})
    return trace

# Run example conversation and print compact human readable trace.
user_message = "What is the weather like today in Denver?"
city = "Denver"
trace = run_with_trace(user_message, city)
for step in trace:
    print(json.dumps(step, indent=2))



### **3.2. Handling tool failures**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_04/Lecture_B/image_03_02.jpg?v=1768769544" width="250">



>* Tools can fail; make failures visible and safe
>* Intentionally trigger edge cases and check error handling

>* Design tools to validate inputs and categorize errors
>* Guide the model to interpret errors and respond safely

>* Use failures to improve multi-step tool workflows
>* Trace cascades, refine rules, and guide recovery



In [ ]:
#@title Python Code - Handling tool failures

# Demonstrate simple tool failures and safe handling in plain Python.
# Show input validation, exception catching, and structured error dictionaries.
# Simulate a model reacting to tool errors with honest user friendly messages.

# !pip install some_required_library_here if external libraries were needed.

# Define a simple weather tool that sometimes fails on purpose.
def weather_tool(city_name, temperature_unit):
    # Validate city name type and length before proceeding.
    if not isinstance(city_name, str) or len(city_name.strip()) < 3:
        return {"ok": False, "error_type": "invalid_city", "message": "City name seems invalid."}

    # Validate temperature unit to accept only Fahrenheit or Celsius.
    if temperature_unit not in ["F", "C"]:
        return {"ok": False, "error_type": "invalid_unit", "message": "Temperature unit must be F or C."}

    # Simulate external system outage for a specific city keyword.
    if city_name.lower().strip() == "atlantis":
        return {"ok": False, "error_type": "not_found", "message": "Requested city data was not found."}

    # Simulate random network failure using simple deterministic condition.
    if len(city_name) % 2 == 0:
        return {"ok": False, "error_type": "network", "message": "Weather service is temporarily unavailable."}

    # Return successful fake temperature when no failure conditions are triggered.
    fake_temperature_fahrenheit = 72
    if temperature_unit == "C":
        fake_temperature_fahrenheit = int((fake_temperature_fahrenheit - 32) * 5 / 9)
    return {"ok": True, "temperature": fake_temperature_fahrenheit, "unit": temperature_unit}

# Define a simple model like wrapper that interprets tool responses.
def model_handle_weather_request(city_name, temperature_unit):
    # Call the weather tool and capture its structured response.
    tool_result = weather_tool(city_name, temperature_unit)

    # If tool succeeded, return a friendly success message.
    if tool_result.get("ok"):
        return f"The temperature in {city_name} is {tool_result['temperature']} degrees {tool_result['unit']}."

    # Handle invalid city error with clarification request.
    if tool_result.get("error_type") == "invalid_city":
        return "I could not understand that city name, please provide a longer valid name."

    # Handle invalid unit error with suggestion for allowed units.
    if tool_result.get("error_type") == "invalid_unit":
        return "That temperature unit is unsupported, please use F for Fahrenheit or C for Celsius."

    # Handle not found error with honest explanation and alternative suggestion.
    if tool_result.get("error_type") == "not_found":
        return "I could not find weather data for that location, please try a nearby real city."

    # Handle network or unknown errors with apology and transparent message.
    return "I am sorry, the weather service failed, please try again later or ask something else."

# Prepare several test scenarios that deliberately trigger different failures.
scenarios = [
    ("NY", "F"),
    ("Atlantis", "F"),
    ("New York", "X"),
    ("Miami", "F"),
    ("LA", "C"),
]

# Run each scenario and print how the model handles tool failures.
for city, unit in scenarios:
    response = model_handle_weather_request(city, unit)
    print(f"User asked weather for {city} in {unit}, model replied: {response}")



### **3.3. Refining Tool Prompts**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master LangChain & Llama 3/Module_04/Lecture_B/image_03_03.jpg?v=1768769579" width="250">



>* Write precise tool names and descriptions for models
>* Clarify when each tool is preferred or avoided

>* Specify when, how, and why tools run
>* Test edge cases, refine prompts for safe behavior

>* Set strict rules for sensitive tool usage
>* Test edge cases and refine safety instructions



# <font color="#418FDE" size="6.5" uppercase>**Tool Calling Basics**</font>


In this lecture, you learned to:
- Explain how LangChain exposes tools for Llama 3 to call during reasoning. 
- Define and register simple tools in LangChain and connect them to a Llama 3 model. 
- Test and debug tool-using interactions to ensure correct tool selection and safe outputs. 

In the next Module (Module 5), we will go over 'Agents And Deployment'